In [ ]:
"""
   Copyright (C) 2023 ETH Zurich. All rights reserved.
   Author: Sergei Vostrikov, ETH Zurich
           Cedric Hirschi, ETH Zurich
   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at
       http://www.apache.org/licenses/LICENSE-2.0
   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

   SPDX-License-Identifier: Apache-2.0
"""

# If you are not familiar with Jupyter Notebooks, please first check online tutorials such as https://realpython.com/jupyter-notebook-introduction/#creating-a-notebook

In [ ]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

### Test of Matplotlib Widget Backend

In [ ]:
import matplotlib.pyplot as plt

plt.plot([[1, 2], [0, 0]])

#### If you do not see a figure above,  try to restart the kernel.
#### If it does not help, check the installation of **ipympl** library.

## Run WULPUS GUI Demo

## Prepare TX/RX configurations for HW

In [ ]:
# from wulpus.rx_tx_conf import WulpusRxTxConfigGen
import wulpus.rx_tx_conf_gui as conf_gui

# # Generate Transmit/Receive configs
# conf_gen = WulpusRxTxConfigGen()

# Generate Transmit/Receive configs using the GUI
conf_gen = conf_gui.WulpusRxTxConfigGenGUI()

display(conf_gen)

In [ ]:
from wulpus.uss_conf import WulpusUssConfig, PGA_GAIN
from wulpus.uss_conf_gui import WulpusUssConfigGUI

# TX and RX active channels IDs (only when not using the GUI)
conf_gen.add_config([7], [7])
# conf_gen.add_config([0], [6])

# load configurations from the GUI
tx_confs = conf_gen.get_tx_configs()
rx_confs = conf_gen.get_rx_configs()

# # load configurations directly from a file
# tx_confs = conf_gen.with_file("tx_rx_configs.json").get_tx_configs()
# rx_confs = conf_gen.with_file("tx_rx_configs.json").get_rx_configs()

# # load configurations for the carotid example
# tx_confs = conf_gen.with_file('examples/carotid/tx_rx_configs_carotid.json').get_tx_configs()
# rx_confs = conf_gen.with_file('examples/carotid/tx_rx_configs_carotid.json').get_rx_configs()

# # load configurations for the waterbath example
# tx_confs = conf_gen.with_file('examples/waterbath/tx_rx_configs_waterbath.json').get_tx_configs()
# rx_confs = conf_gen.with_file('examples/waterbath/tx_rx_configs_waterbath.json').get_rx_configs()

print('TX config: ', np.binary_repr(tx_confs[0]))
print('RX config: ', np.binary_repr(rx_confs[0]))

# Create US subsystem configuration
uss_conf = WulpusUssConfig(num_acqs=2000,
                        #    dcdc_turnon=195315,
                        #    start_hvmuxrx=500,
                        #    meas_period=228885,
                           num_txrx_configs=len(tx_confs),
                           tx_configs=tx_confs,
                           rx_configs=rx_confs,
                           rx_gain=PGA_GAIN[16])
                        #    trans_freq=1000000,
                        #    num_pulses=10,
                        #    sampling_freq=4000000)

# Modify US subsystem configuration using the GUI
uss_conf = WulpusUssConfigGUI(uss_conf)

# # Load settings from a file
# uss_conf.with_file('uss_config.json')

# # Load settings from a file for the carotid example
# uss_conf.with_file('examples/carotid/uss_config_carotid.json')

# # Load settings from a file for the waterbath example
# uss_conf.with_file('examples/waterbath/uss_config_waterbath.json')

print("Gain dB: ", uss_conf.rx_gain)

display(uss_conf)

### Initialize a connection

We initialize a `WulpusDongle` object here such that it can run independently from the GUI.

In [ ]:
from wulpus.dongle import WulpusDongle

# Create a dongle object
dongle = WulpusDongle()

### Run GUI 
(Check sw/docs/gui_overview.pdf for more information)

In [ ]:
%matplotlib widget
from wulpus.gui import WulpusGuiSingleCh

# Create a GUI
gui = WulpusGuiSingleCh(dongle, uss_conf, max_vis_fps = 20)

display(gui)

## Loading and extracting the saved data

The data is saved in a `.npz` file. This is a compressed file format that can be used with the `numpy` library.

In [ ]:
# Data can be loaded as easy as:
data = np.load('examples/waterbath/data_31.npz')

# The keys of the data are:
print('Keys:', data.files)

The structure is a dictionary with the length `num_acquisitions` (Number of acquisitions) and consists of the following keys:

**data_arr:** This column actually contains the data, each of length `num_samples` (Number of samples). This can be seen as data for one A-mode diagram.

**acq_num_arr:** The number of each acquisition. The acquisition number is incremented by 1 for each acquisition, starting at zero. If one number is missing (step larger than one), it means that that this acquisition got dropped during transmission.

**tx_rx_id_arr:** The TX/RX configuration ID of the acquisition. These are the same as the ones in the `Active RX ID` dropdown in the GUI or how they are saved from the TX/RX configuration GUI.

In [ ]:
# The shape of the data is:
print('Data shape:', data['data_arr'].shape)

# The unique IDs of the active channels are:
print('Unique IDs:', np.unique(data['tx_rx_id_arr']))

In the case of this waterbath example, we have 100 acquisitions, each with 400 samples. The TX/RX configuration is the same for all acquisitions (0).

Since the `data_arr` is a bit awkward to use in this form (`(400, 100)`), we transpose it to `(100, 400)`.

In [ ]:
# Transpose the data such that we can index via time
data_meas = data['data_arr'].T

# The shape of the data is:
print('Data shape:', data_meas.shape)

The data can then be easily used further, for example to just make a plot of one acquisition:

In [ ]:
# Plot one acquisition
FRAME = 50

plt.figure(figsize=(10, 5))
plt.plot(data_meas[FRAME])
plt.title(f'Acquisition {FRAME}/{len(data_meas)} of the waterbath example')
plt.xlabel('Samples')
plt.ylabel('ADC digital code')
plt.grid()
plt.show()

**Note:** In this example, we have large oscillations which is due to the nature of the transducer.

Something more advanced and useful can be done with `ipywidgets`: Make a slider to go through the acquisitions:

In [ ]:
plt.figure(figsize=(10, 5))
plot_data = plt.plot(data_meas[0])
plt.ylim(-2500, 2500)
plt.title(f'Acquisition 0/{len(data_meas)} of the waterbath example')
plt.xlabel('Samples')
plt.ylabel('ADC digital code')
plt.grid()
plt.show()

def visualize(frame):
    plot_data[0].set_ydata(data_meas[frame])
    plt.title(f'Acquisition {frame}/{len(data_meas)} of the waterbath example')
    plt.draw()

widgets.interact(visualize, frame=widgets.IntSlider(min=0, max=len(data_meas)-1, step=1, value=0))

We can see that during the first ten samples, the object is moved further from the transducer. This can be observed via the smaller, high frequency oscillations.